In [3]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer


In [5]:
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import numpy as np

class PoliticalDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(label, dtype=torch.long)
        }

def calculate_metrics(preds, labels):
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return accuracy, precision, recall, f1

df = pd.read_csv('preprocessed_final_dataset.csv')
# df = df.sample(n=100, random_state=42)

label_map = {'left': 0, 'right': 1, 'center': 2}
df['Leaning'] = df['Leaning'].map(label_map)

batch_size = 32

train_texts, val_texts, train_labels, val_labels = train_test_split(df['Text'].tolist(), df['Leaning'].tolist(), test_size=0.1)

train_dataset = PoliticalDataset(train_texts, train_labels, tokenizer, max_len=128)
val_dataset = PoliticalDataset(val_texts, val_labels, tokenizer, max_len=128)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)




In [6]:
print(df['Leaning'].unique())


[2 1 0]


In [7]:
import torch
from transformers import BertTokenizer
from torch.nn import functional as F

def classify_text(input_text, model, device='cuda'):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    model.eval()

    encoding = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs

        preds = torch.argmax(logits, dim=1).item()

    label_map_inv = {0: 'left', 1: 'right', 2: 'center'}
    predicted_label = label_map_inv[preds]

    return predicted_label


def is_biased(input_text, model):
    predicted_label = classify_text(input_text, model)
    return predicted_label != 'center'

In [1]:
!pip install Dbias
from Dbias.bias_classification import *

# returns classification label for a given sentence fragment.
classifier("Nevertheless, Trump and other Republicans have tarred the protests as havens for terrorists intent on destroying property.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/657 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
Device set to use 0


[{'label': 'Biased', 'score': 0.9938021898269653}]

In [ ]:
my_model_predictions = [is_biased(text, model) for text in val_texts]

In [ ]:
my_model_predictions[4]

False

In [8]:
# Extract Dbias bias predictions (True/False)
dbias_predictions = [classifier(text) for text in val_texts]

In [9]:
dbias_predictions[4]

[{'label': 'Biased', 'score': 0.9126459360122681}]

In [10]:
dbias_predictions_bool = [pred[0]['label'] == 'Biased' for pred in dbias_predictions]

In [11]:
for i in range(5):
  print(f"{dbias_predictions[i]} and {dbias_predictions_bool[i]}")

[{'label': 'Non-biased', 'score': 0.7160286903381348}] and False
[{'label': 'Biased', 'score': 0.8370468616485596}] and True
[{'label': 'Biased', 'score': 0.6641389727592468}] and True
[{'label': 'Non-biased', 'score': 0.5940608382225037}] and False
[{'label': 'Biased', 'score': 0.9126459360122681}] and True


In [12]:
val_labels[:5]

[0, 1, 0, 0, 1]

In [13]:
val_binary_labels = [False if label == 2 else True for label in val_labels]

In [14]:
# acc_mine = accuracy_score(val_binary_labels, my_model_predictions)
acc_theirs = accuracy_score(val_binary_labels, dbias_predictions_bool)

# print(f"mine={acc_mine} theirs={acc_theirs}")
print(f"dbias acc={acc_theirs}")

dbias acc=0.718645219755324
